In [1]:
# 数据预处理
import numpy as np
import scipy.sparse as sp
import torch

def encode_onehot(labels):
    """
    将类别表示为one-hot形式的矩阵表示
    """
    classes = set(labels)
    classes_dict = {
        c:np.identity(len(classes))[i,:] for i,c in enumerate(classes)
    }
    """
    >>> a = {"a":1,"b":2}
    >>> labels = ['a','b','a','a']
    >>> list(map(a.get,labels))
    [1, 2, 1, 1]
    """
    # 标签类别转换位onehot形式
    labels_onehot = np.array(list(map(classes_dict.get,labels)),dtype=np.int32)
    return labels_onehot

def normalize(mx):
    """
    首先对每一行求和得到rowsum；求倒数得到r_inv；
    如果某一行全为0，则r_inv算出来会等于无穷大，将这些行的r_inv置为0；
    构建对角元素为r_inv的对角的元素；
    用对角矩阵与原始矩阵的点积起到标准化的作用，原始矩阵中每一行元素都会与对应的r_inv相乘。
    """
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx
  
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

# 加载数据
def load_data(path, dataset):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)
    
    # 邻接矩阵；特征；类别；训练集合测试集
    return adj, features, labels, idx_train, idx_val, idx_test


load_data(path="./data/cora/",dataset="cora")

Loading cora dataset...


(tensor(indices=tensor([[   0,    8,   14,  ..., 1389, 2344, 2707],
                        [   0,    0,    0,  ..., 2707, 2707, 2707]]),
        values=tensor([0.1667, 0.1667, 0.0500,  ..., 0.2000, 0.5000, 0.2500]),
        size=(2708, 2708), nnz=13264, layout=torch.sparse_coo),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([0, 4, 5,  ..., 3, 6, 0]),
 tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
          28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
          42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
          56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
          70,  71,  72,  73

In [2]:
# 图卷积
import torch,math
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

class GraphConvolution(torch.nn.Module):
    
    """
    GCN Layer
    """
    
    def __init__(self,in_features,out_features,bias=True):
        super(GraphConvolution,self).__init__()
        self.in_featues = in_features
        self.out_features = out_features
        
        """
        torch.nn.parameter.Parameter(torch.FloatTensor(hidden_size))：
        Parameter是Tensor，即 Tensor 拥有的属性它都有，⽐如可以根据data 来访问参数数值，⽤ grad 来访问参数梯度。
       
       
        register_parameter：向建立的网络module添加 parameter；最大的区别：parameter可以通过注册网络时候的name获取。
       
       """
        self.weight = Parameter(torch.FloatTensor(in_features,out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias',None)
            
        # 初始化参数
        self.reset_parameters()
        
    def reset_parameters(self,):
        stdv = 1. / math.sqrt(self.weight.size(1))
        # 使用均匀分布来初始化其权重
        self.weight.data.uniform_(-stdv,stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv,stdv)
            
    def forward(self,input,adj):
        # AXW
        ## XW 
        support = torch.mm(input,self.weight) # torch.mm：矩阵相乘
        ## A(XW)
        output = torch.spmm(adj,support) # 稀疏矩阵相乘
        
        if self.bias is not None:
            return output + self.bias
        else:
            return output
        
    def __repr(self,):
        return "{}({}->{})".format(self.__class__.__name__,str(self.in_features),str(self.out_features)) 

In [3]:
import torch.nn.functional as F

class GCN(torch.nn.Module):
    
    def __init__(self,nfeat,nhid,nclass,dropout):
        """
        nfeat：特征数
        nhid：隐藏层大小
        nclass：目标维度
        """
        super(GCN,self).__init__()
        self.gc1 = GraphConvolution(nfeat,nhid)
        self.gc2 = GraphConvolution(nhid,nclass)
        self.dropout = dropout
        
        
    def forward(self,x,adj):
        x = F.relu(self.gc1(x,adj))
        # 带Dropout的网络可以防止出现过拟合。
        x = F.dropout(x,self.dropout,training=self.training)
        # 列的和为1
        x = F.log_softmax(self.gc2(x,adj),dim=1)
        return x

In [4]:
# 准确率
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

In [5]:
# Training settings

from types import SimpleNamespace
import torch

args = {
    'seed':42,
    'no_cuda':False,
    'fastmode':False, # Validate during training pass.
    'epochs':200, # 步长
    'lr':0.01, # 学习率
    'weight_decay':5e-4, # 权重衰减（L2惩罚）（默认: 0）
    'hidden':16, # 隐藏层
    'dropout':0.5,
}
# 将字典转换为对象
args = SimpleNamespace(**args)
# 检查cuda是否可用
args.cuda = not args.no_cuda and torch.cuda.is_available()
args

namespace(seed=42,
          no_cuda=False,
          fastmode=False,
          epochs=200,
          lr=0.01,
          weight_decay=0.0005,
          hidden=16,
          dropout=0.5,
          cuda=True)

In [6]:
np.random.seed(args.seed)
torch.manual_seed(args.seed) #为CPU设置种子用于生成随机数，以使得结果是确定的
if args.cuda: #为当前GPU设置随机种子；如果使用多个GPU，应该使用torch.cuda.manual_seed_all()为所有的GPU设置种子。
    torch.cuda.manual_seed(args.seed)

In [7]:
# Load data
adj, features, labels, idx_train, idx_val, idx_test = load_data(path="./data/cora/",dataset="cora")

Loading cora dataset...


In [8]:
# 模型和优化器
model = GCN(nfeat=features.shape[1],
            nhid=args.hidden,nclass=labels.max().item()+1,
            dropout=args.dropout)

optimizer = torch.optim.Adam(model.parameters(),
                      lr=args.lr,weight_decay=args.weight_decay)

In [9]:
if args.cuda: # 对model自身进行的内存迁移
    """
    model = model.cuda() 
    <=>
    model.cuda() 
    """
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()

In [10]:
import time
import torch.nn.functional as F
def train(epoch):
    t = time.time()
    """
    model.eval()，pytorch会自动把BN和DropOut固定住，不会取平均，而是用训练好的值。不然的话，一旦test的batch_size过小，很容易就会被BN层导致生成图片颜色失真极大；在模型测试阶段使用

    model.train() 让model变成训练模式，此时 dropout和batch normalization的操作在训练q起到防止网络过拟合的问题
    """
    model.train()
    optimizer.zero_grad()
    output = model(features,adj)
    loss_train = F.nll_loss(output[idx_train],labels[idx_train])
    acc_train = accuracy(output[idx_train],labels[idx_train])
    loss_train.backward()
    optimizer.step()
    
    
    if not args.fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)
    
    loss_val = F.nll_loss(output[idx_val],labels[idx_val])
    acc_val = accuracy(output[idx_val],labels[idx_val])
    
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

In [11]:
t_total = time.time()
for epoch in range(args.epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

Epoch: 0001 loss_train: 1.9926 acc_train: 0.1357 loss_val: 1.9962 acc_val: 0.1567 time: 0.8208s
Epoch: 0002 loss_train: 1.9796 acc_train: 0.1857 loss_val: 1.9841 acc_val: 0.1567 time: 0.0068s
Epoch: 0003 loss_train: 1.9645 acc_train: 0.2071 loss_val: 1.9729 acc_val: 0.1567 time: 0.0068s
Epoch: 0004 loss_train: 1.9584 acc_train: 0.2000 loss_val: 1.9620 acc_val: 0.1567 time: 0.0068s
Epoch: 0005 loss_train: 1.9488 acc_train: 0.2000 loss_val: 1.9513 acc_val: 0.1567 time: 0.0078s
Epoch: 0006 loss_train: 1.9324 acc_train: 0.2000 loss_val: 1.9408 acc_val: 0.1567 time: 0.0068s
Epoch: 0007 loss_train: 1.9135 acc_train: 0.2000 loss_val: 1.9305 acc_val: 0.1567 time: 0.0068s
Epoch: 0008 loss_train: 1.9082 acc_train: 0.2000 loss_val: 1.9202 acc_val: 0.1567 time: 0.0068s
Epoch: 0009 loss_train: 1.9026 acc_train: 0.2000 loss_val: 1.9096 acc_val: 0.1567 time: 0.0078s
Epoch: 0010 loss_train: 1.8910 acc_train: 0.2000 loss_val: 1.8990 acc_val: 0.1567 time: 0.0068s
Epoch: 0011 loss_train: 1.8782 acc_train

Epoch: 0095 loss_train: 0.8406 acc_train: 0.8714 loss_val: 1.0106 acc_val: 0.7900 time: 0.0068s
Epoch: 0096 loss_train: 0.8101 acc_train: 0.8357 loss_val: 1.0014 acc_val: 0.7933 time: 0.0068s
Epoch: 0097 loss_train: 0.7616 acc_train: 0.8714 loss_val: 0.9934 acc_val: 0.7900 time: 0.0068s
Epoch: 0098 loss_train: 0.7803 acc_train: 0.8500 loss_val: 0.9867 acc_val: 0.7900 time: 0.0088s
Epoch: 0099 loss_train: 0.7967 acc_train: 0.8071 loss_val: 0.9807 acc_val: 0.7900 time: 0.0068s
Epoch: 0100 loss_train: 0.7344 acc_train: 0.8643 loss_val: 0.9735 acc_val: 0.7933 time: 0.0064s
Epoch: 0101 loss_train: 0.7825 acc_train: 0.8500 loss_val: 0.9669 acc_val: 0.7967 time: 0.0068s
Epoch: 0102 loss_train: 0.7703 acc_train: 0.8357 loss_val: 0.9608 acc_val: 0.7933 time: 0.0078s
Epoch: 0103 loss_train: 0.7129 acc_train: 0.8714 loss_val: 0.9537 acc_val: 0.7933 time: 0.0059s
Epoch: 0104 loss_train: 0.6960 acc_train: 0.8714 loss_val: 0.9467 acc_val: 0.7933 time: 0.0078s
Epoch: 0105 loss_train: 0.7183 acc_train

Epoch: 0191 loss_train: 0.4250 acc_train: 0.9571 loss_val: 0.7061 acc_val: 0.8100 time: 0.0068s
Epoch: 0192 loss_train: 0.4002 acc_train: 0.9571 loss_val: 0.7051 acc_val: 0.8100 time: 0.0068s
Epoch: 0193 loss_train: 0.4082 acc_train: 0.9286 loss_val: 0.7059 acc_val: 0.8133 time: 0.0059s
Epoch: 0194 loss_train: 0.3701 acc_train: 0.9214 loss_val: 0.7082 acc_val: 0.8167 time: 0.0059s
Epoch: 0195 loss_train: 0.3871 acc_train: 0.9571 loss_val: 0.7115 acc_val: 0.8100 time: 0.0068s
Epoch: 0196 loss_train: 0.3849 acc_train: 0.9571 loss_val: 0.7125 acc_val: 0.8067 time: 0.0068s
Epoch: 0197 loss_train: 0.4187 acc_train: 0.9286 loss_val: 0.7109 acc_val: 0.8067 time: 0.0068s
Epoch: 0198 loss_train: 0.3656 acc_train: 0.9500 loss_val: 0.7073 acc_val: 0.8100 time: 0.0069s
Epoch: 0199 loss_train: 0.3828 acc_train: 0.9429 loss_val: 0.7044 acc_val: 0.8100 time: 0.0059s
Epoch: 0200 loss_train: 0.4004 acc_train: 0.9286 loss_val: 0.7007 acc_val: 0.8100 time: 0.0059s
Optimization Finished!
Total time elapse

In [12]:
def test():
    model.eval()
    output = model(features,adj)
    loss_test = F.nll_loss(output[idx_test],labels[idx_test])
    acc_test = accuracy(output[idx_test],labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))
test()

Test set results: loss= 0.7228 accuracy= 0.8380
